## ADALINE
パーセプトロンとの違いは、重みの更新方法
* パーセプトロン：単位ステップ関数
* ADALINE：線形活性化関数 \\(\phi(z)\\)

目的関数(Objective function) ... 学習過程で最適化される関数。多くの場合は ** コスト関数 ** (cost function)

## コスト関数
誤差平方和(Sum of Squared Error：SSE)
$$ J(w) = \frac{1}{2}\sum_i(y^{(i)}-\phi(z^{(i)}))^2 $$

利点
* 微分可能である
* 凸関数であるため勾配降下法(gradient descent)を用いてコスト関数を最小化する重みを見つけ出すことができる。

### 勾配降下法を使った重み更新
コスト関数\\( J(w) \\)の勾配\\( \nabla J(w) \\)に沿って1ステップ進む
$$ w := w + \Delta w $$
重みの変化である\\( \Delta w \\)は、負の勾配に学習率\\( \eta \\)を掛けたもの
$$ \Delta w = -\eta\nabla J(w) $$

### 勾配計算(偏微分係数)
$$ \begin{align} \frac{\delta J}{\delta w_j} = \frac{\delta}{\delta w_j}\frac{1}{2}\sum_i(y^{(i)}-\phi(z^{(i)}))^2 \\
= \frac{1}{2}\frac{\delta}{\delta w_j}\sum_i(y^{(i)}-\phi(z^{(i)}))^2 \\
= \frac{1}{2}\sum_i2(y^{(i)}-\phi(z^{(i)}))\frac{\delta}{\delta w_j}(y^{(i)}-\phi(z^{(i)})) \\
= \sum_i(y^{(i)}-\phi(z^{(i)}))\frac{\delta}{\delta w_j}\Bigl( y^{(i)}-\sum_k(w_kx_k^{(i)})\Bigr) \\
\end{align} $$